In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import scipy

In [ ]:
from scipy import stats
import statsmodels.formula.api as smf
from statsmodels.graphics.regressionplots import influence_plot


In [ ]:
corolla = pd.read_csv('toyoto_corrola.csv')

In [ ]:
corolla.head()

In [ ]:
corolla.info()

In [ ]:
corolla = corolla.drop(['Model','Id'],axis=1)

In [ ]:
corolla.head()

In [ ]:
corolla.isna().sum()

In [ ]:
corolla.corr()

In [ ]:
corolla.info()

In [ ]:
sns.set_style(style='darkgrid')
sns.pairplot(corolla)

In [ ]:
corolla.tail()

In [ ]:
corolla = corolla.rename({'Age_08_04':'Age'},axis=1)


In [ ]:
corolla.head()

In [ ]:
corolla[corolla.duplicated()]

In [ ]:
corolla = corolla.drop_duplicates().reset_index(drop=True)

In [ ]:
corolla[corolla.duplicated()]

In [ ]:
#Building a model

import statsmodels.formula.api as smf
model = smf.ols('Price~Age+KM+HP+Doors+Cylinders+Gears+Weight',data=corolla).fit()

In [ ]:
model.params

In [ ]:
print(model.tvalues,'\n',np.round(model.pvalues,5))

In [ ]:
(model.rsquared,model.rsquared_adj) #Current model accuracy is 86.15%

In [ ]:
# Build SLR and MLR models for insignificant variable 'Doors'(it's p value is greater then 0.05)
# Also find their tvalues and pvalues

# Regresiion models


In [ ]:
model_d = smf.ols('Price~Doors',data=corolla).fit()
print(model_d.tvalues,model_d.pvalues)

# Model Validation Techniques
Two Techniques: 1. Collinearity Check & 2. Residual Analysis

In [ ]:
#Collinearity problem check
#Calculate VIF = 1/(1-Rsquare) for all independent variable

rsq_hp = smf.ols('HP~KM+Age+Doors+Cylinders+Gears+Weight',data=corolla).fit().rsquared
vif_hp = 1/(1-rsq_hp)

rsq_km = smf.ols('KM~Age+HP+Doors+Cylinders+Gears+Weight',data=corolla).fit().rsquared
vif_km = 1/(1-rsq_km)

rsq_age = smf.ols('Age~KM+HP+Doors+Cylinders+Gears+Weight',data=corolla).fit().rsquared
vif_age = 1/(1-rsq_age)

rsq_doors = smf.ols('Doors~KM+HP+Age+Cylinders+Gears+Weight',data=corolla).fit().rsquared
vif_doors = 1/(1-rsq_doors)

rsq_cyl = smf.ols('Cylinders~KM+HP+Doors+Age+Gears+Weight',data=corolla).fit().rsquared
vif_cyl = 1/(1-rsq_cyl)

rsq_gears = smf.ols('Gears~KM+HP+Doors+Cylinders+Age+Weight',data=corolla).fit().rsquared
vif_gears = 1/(1-rsq_gears)

rsq_weight = smf.ols('Weight~KM+HP+Doors+Cylinders+Gears+Age',data=corolla).fit().rsquared
vif_weight = 1/(1-rsq_weight)

d1 = {'Variables':['HP','Age','KM','Weight','Cylinders','Gears','Doors'],'VIF':[vif_age,vif_km,vif_hp,vif_cyl,vif_gears,vif_weight,vif_doors]}
Vif_frame = pd.DataFrame(d1)  
Vif_frame


In [ ]:
# None variable has VIF>20, No Collinearity, so consider all varaibles in Regression equation

In [ ]:
#Residual Analysis
# Test for Normality of Residuals (Q-Q Plot) using residual model (model.resid)

import statsmodels.api as sm
qqplot = sm.qqplot(model.resid,line='q')
plt.title('Normal Q-Q plot of residuals')
plt.show()

In [ ]:
list(np.where(model.resid>6000)) #Outlier detection from Q-Q plot

In [ ]:
list(np.where(model.resid<-6000))

In [ ]:
# Test for Homoscedasticity or Heteroscedasticity (plotting model's standardized fitted values vs standardized residual

def get_standardized_values(vals):
    return (vals - vals.mean())/vals.std()

In [ ]:
plt.scatter(get_standardized_values(model.fittedvalues),get_standardized_values(model.resid))

plt.title('Residual plot')
plt.xlabel('Standardized Fitted Values')
plt.ylabel('Standardized residual values')
plt.show()

In [ ]:
# Test for errors or Residuals Vs Regressors or independent 'x' variables or predictors 
# using Residual Regression Plots code graphics.plot_regress_exog(model,'x',fig)    # exog = x-variable & endog = y

In [ ]:
fig = plt.figure(figsize=(15,8))
fig = sm.graphics.plot_regress_exog(model,"Age",fig=fig)
plt.show()

In [ ]:
fig = plt.figure(figsize=(15,8))
fig = sm.graphics.plot_regress_exog(model,"KM",fig=fig)
plt.show()

In [ ]:
fig = plt.figure(figsize=(15,8))
fig = sm.graphics.plot_regress_exog(model,'HP',fig=fig)
plt.show()

In [ ]:
fig = plt.figure(figsize=(15,8))
fig = sm.graphics.plot_regress_exog(model,'Doors',fig=fig)
plt.show()

In [ ]:
fig = plt.figure(figsize=(15,8))
fig = sm.graphics.plot_regress_exog(model,'Cylinders',fig=fig)
plt.show()

In [ ]:
fig = plt.figure(figsize=(15,8))
fig = sm.graphics.plot_regress_exog(model,'Gears',fig=fig)
plt.show()

In [ ]:
fig = plt.figure(figsize=(15,8))
fig = sm.graphics.plot_regress_exog(model,'Weight',fig=fig)
plt.show()

# Model Deletion Diagnostics (checking Outliers or Influencers)
Two Techniques : 1. Cook's Distance & 2. Leverage value

In [ ]:
model_influence = model.get_influence()
(c, _) = model_influence.cooks_distance


In [ ]:
fig  = plt.subplots(figsize=(20,7))
plt.stem(np.arange(len(corolla)),np.round(c,3))
plt.xlabel('Row index')
plt.ylabel('Cooks Distance')
plt.show()

In [ ]:
# Index and value of influencer where C>0.5
(np.argmax(c),np.max(c))

In [ ]:
from statsmodels.graphics.regressionplots import influence_plot
influence_plot(model)
plt.show()

In [ ]:
k  = corolla.shape[1]
n  = corolla.shape[0]
leverage_cutoff = 3*((k+1)/n)


In [ ]:
leverage_cutoff

In [ ]:
corolla[corolla.index.isin([220])]

# Improving the model

In [ ]:
corolla_new = corolla.copy()
corolla_new

In [ ]:
df = corolla_new.drop(corolla_new.index[[220]],axis=0).reset_index()

In [ ]:
df.head()

In [ ]:
df = df.drop(['index'],axis=1)

In [ ]:
df.head()

# Model Deletion Diagnostics and Final Model

In [ ]:
while model.rsquared < 0.90:
    for c in [np.max(c)>0.5]:
        model=smf.ols('Price~Age+KM+HP+Doors+Cylinders+Gears+Weight',data=df).fit()
        (c,_)=model.get_influence().cooks_distance
        c
        np.argmax(c) , np.max(c)
        df=df.drop(df.index[[np.argmax(c)]],axis=0).reset_index(drop=True)
        df
    else:
        final_model=smf.ols('Price~Age+KM+HP+Doors+Cylinders+Gears+Weight',data=df).fit()
        final_model.rsquared , final_model.aic
        print("Thus model accuracy is improved to",final_model.rsquared)

In [ ]:
new_data = pd.DataFrame({'Age':71,'KM':17016,'HP':86,'Doors':3,'Cylinders':4,'Gears':5,'Weight':1015},index=[1])

In [ ]:
new_data

In [ ]:
final_model.predict(new_data)

In [ ]:
pred_x = final_model.predict(df)

In [ ]:
pred_x

In [ ]:
df.head()

In [ ]:
df.tail()